In [ ]:
import tensorflow as tf
import torch
from torchsummary import summary

In [2]:
def add_conv2d_layer(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    """
    Adds a Conv2D layer to the model.
    
    Args:
        in_channels: Number of input channels.
        out_channels: Number of output channels.
        kernel_size: Size of the convolution kernel.
        stride: Stride of the convolution.
        padding: Padding added to all four sides of the input.
    """
    return f"torch.nn.Conv2d(in_channels={in_channels}, out_channels={out_channels}, kernel_size={kernel_size}, stride={stride}, padding={padding})"

In [3]:
def add_Linear_layer(in_features, out_features):
    """
    Adds a Linear layer to the model.
    
    Args:
        in_features: Number of input features.
        out_features: Number of output features.
    """
    return f"torch.nn.Linear(in_features={in_features}, out_features={out_features})"

In [4]:
def add_dropout_layer(p=0.5):
    """
    Adds a Dropout layer to the model.
    
    Args:
        p: Probability of an element to be zeroed.
    """
    return f"torch.nn.Dropout(p={p})"

In [5]:
def add_Upsample2d_layer(scale_factor=2, mode='nearest'):
    """
    Adds an Upsample layer to the model.
    
    Args:
        scale_factor: The factor by which to upsample the input.
        mode: The algorithm used for upsampling.
    """
    return f"torch.nn.Upsample(scale_factor={scale_factor}, mode='{mode}')"

In [6]:
def add_relu_layer(convolutional_layer):
    """
    Adds a ReLU activation layer to the model.
    """
    return f"torch.nn.functional.relu(input={convolutional_layer})"

In [7]:
def add_maxpool2d_layer(convolutional_layer,kernel_size=2, stride=2):
    """
    Adds a MaxPool2d layer to the model.
    
    Args:
        kernel_size: Size of the window to take a max over.
        stride: Stride of the window.
    """
    return f"torch.nn.functional.max_pool2d(input={convolutional_layer}, kernel_size={kernel_size}, stride={stride})"

In [8]:
def add_concat_layer(tensor1, tensor2, dim=1):
    """
    Adds a Concatenate layer to the model.
    
    Args:
        tensor1: First tensor to concatenate.
        tensor2: Second tensor to concatenate.
        dim: Dimension along which to concatenate.
    """
    return f"torch.cat([{tensor1}, {tensor2}], dim={dim})"

In [9]:
add_conv2d_layer( None, None, kernel_size=3, stride=1, padding=1)

'torch.nn.Conv2d(in_channels=None, out_channels=None, kernel_size=3, stride=1, padding=1)'

In [5]:
class Parameters:
    def __init__(self, layer):
        self.layer = layer
        print(f"Parameters initialized for layer: {self.layer.id}")
    def get_params(self):
        return {
            'layer': self.layer.layer
        }        


In [6]:
class Conv2DParams(Parameters):
    def __init__(self, layer, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__(layer)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
    
    def get_params(self):
        return {
            'layer': self.layer.layer,
            'in_channels': self.in_channels,
            'out_channels': self.out_channels,
            'kernel_size': self.kernel_size,
            'stride': self.stride,
            'padding': self.padding
        }

In [40]:
class MaxPool2DParams(Parameters):
    def __init__(self, layer, kernel_size=2, stride=2):
        super().__init__(layer)
        self.kernel_size = kernel_size
        self.stride = stride
    
    def get_params(self):
        return {
            'layer': self.layer.layer,
            'kernel_size': self.kernel_size,
            'stride': self.stride
        }

In [ ]:
class Layer:
    input_id_counter = 0
    conv2d_id_counter = 0
    maxpool2d_id_counter = 0
    linear_id_counter = 0
    dropout_id_counter = 0
    upsample2d_id_counter = 0
    relu_id_counter = 0
    concat_id_counter = 0
    
    def __init__(self, layer_name, params=None):
        
        match layer_name:
            case 'Input':
                self.id = 'x'
                Layer.input_id_counter += 1
            case 'Conv2D':
                self.id = 'Conv2D_'+str(Layer.conv2d_id_counter)
                Layer.conv2d_id_counter += 1
            case 'MaxPool2D':
                self.id = "MaxPool2D_"+str(Layer.maxpool2d_id_counter)
                Layer.maxpool2d_id_counter += 1
            case 'Linear':
                self.id = "Linear_"+ str(Layer.linear_id_counter)
                Layer.linear_id_counter += 1
            case 'Dropout':
                self.id = "dropout2d_"+ str(Layer.dropout_id_counter)
                Layer.dropout_id_counter += 1
            case 'Upsample2D':
                self.id = "upsample2d_"+ str(Layer.upsample2d_id_counter)
                Layer.upsample2d_id_counter += 1
            case 'ReLU':
                self.id = "relu_"+ str(Layer.relu_id_counter)
                Layer.relu_id_counter += 1
            case 'Concat':
                self.id = "concat_"+ str(Layer.concat_id_counter)
                Layer.concat_id_counter += 1
            case _:
                raise ValueError(f"Unknown layer type: {layer_name}")
        self.layer = self.id.split('_')[0]
        self.params = params
        
    def get_params(self):
        return {
            'id': self.id,
            'layer': self.layer,
            'params': self.params.get_params() if self.params else {}
        }
    

In [42]:
x = torch.zeros((3, 224, 224))

In [2]:
input = Layer('Input')

In [3]:
input.id

'x'

In [17]:
Layer('Conv2D', Conv2DParams(Layer('Input'), in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)).get_params()

Parameters initialized for layer: x


{'id': 'conv2d_4',
 'layer': 'conv2d',
 'params': {'layer': 'x',
  'in_channels': 3,
  'out_channels': 64,
  'kernel_size': 3,
  'stride': 1,
  'padding': 1}}

In [37]:
Conv2DParams(input, in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1).get_params()

Parameters initialized for layer: 0


{'layer': 0,
 'in_channels': 3,
 'out_channels': 64,
 'kernel_size': 3,
 'stride': 1,
 'padding': 1}

In [207]:
queue = [{"id": "conv2d_1", "layer": "conv2d", "params": {"layer": "x", "in_channels": 3, "out_channels": 64, "kernel_size": 3, "stride": 1, "padding": 1}}, {"layer": "relu", "id": "relu_1", "params":{"layer": "conv2d_1"}}, {"layer": "maxpool2d", "id": "maxpool2d_1", "params":{"layer": "relu_1", "kernel_size": 2, "stride": 2}}, {"id": "conv2d_2", "layer": "conv2d", "params": {"layer": "maxpool2d_1", "in_channels": 64, "out_channels": 128, "kernel_size": 3, "stride": 1, "padding": 1}}, {"layer": "relu", "id": "relu_2", "params":{"layer": "conv2d_2"}}, {"layer": "maxpool2d", "id": "maxpool2d_2", "params":{"layer": "relu_2", "kernel_size": 2, "stride": 2}}]

In [208]:
model_name = "Model"

In [209]:
model_class_str = f"class {model_name}(torch.nn.Module):\n"+"\tdef __init__(self):\n" + "\t\tsuper().__init__()\n"

In [210]:
print(model_class_str)

class Model(torch.nn.Module):
	def __init__(self):
		super().__init__()



In [211]:
forward_str = "\tdef forward(self, x):\n"

In [212]:
conv2d_layers = []
relu_layers = []
maxpool2d_layers = []
for i, layer in enumerate(queue):
    match layer["layer"]:
        case "conv2d":
            conv2d_layers.append(layer['id'])
            params = layer["params"]
            model_class_str += f"\t\tself.{layer['id']} = {add_conv2d_layer(params['in_channels'], params['out_channels'], params['kernel_size'], params['stride'], params['padding'])}\n"
            forward_str += f"\t\t{layer['id']} = "+f"self.{layer['id']}"+  f"({params['layer']})\n"
        case "relu":
            relu_layers.append(layer['id'])
            params = layer["params"]
            forward_str += f"\t\t{layer['id']} = {add_relu_layer(params['layer'])}\n"
        case "maxpool2d":
            maxpool2d_layers.append(layer['id'])
            params = layer["params"]
            forward_str += f"\t\t{layer['id']} = {add_maxpool2d_layer(params['layer'], params['kernel_size'], params['stride'])}\n"
model_class_str+= '\n'+forward_str
model_class_str += f"\t\treturn {queue[-1]['id']}\n"

In [214]:
print(model_class_str)

class Model(torch.nn.Module):
	def __init__(self):
		super().__init__()
		self.conv2d_1 = torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
		self.conv2d_2 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

	def forward(self, x):
		conv2d_1 = self.conv2d_1(x)
		relu_1 = torch.nn.functional.relu(input=conv2d_1)
		maxpool2d_1 = torch.nn.functional.max_pool2d(input=relu_1, kernel_size=2, stride=2)
		conv2d_2 = self.conv2d_2(maxpool2d_1)
		relu_2 = torch.nn.functional.relu(input=conv2d_2)
		maxpool2d_2 = torch.nn.functional.max_pool2d(input=relu_2, kernel_size=2, stride=2)
		return maxpool2d_2



In [220]:
exec(model_class_str)

In [221]:
model = eval(f"{model_name}()")

In [222]:
summary(model, input_size=(3, 224, 224), batch_size=1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 224, 224]           1,792
            Conv2d-2         [1, 128, 112, 112]          73,856
Total params: 75,648
Trainable params: 75,648
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 36.75
Params size (MB): 0.29
Estimated Total Size (MB): 37.61
----------------------------------------------------------------
